In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import catboost as cb
from catboost import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
import time
import os

In [2]:
new_data = pd.read_csv('./private_2_processed.csv')
sub = pd.read_csv('./datasets/private_2_template_v2.csv',index_col=0)


In [3]:
model = cb.CatBoostClassifier()
model.load_model('./pretrain_model/base_model_with_public.cbm')

In [4]:
pred_1 = model.predict_proba(new_data[model.feature_names_])[:,1]
new_data['pred_1'] = pred_1
pd.DataFrame(pred_1.round()).value_counts()

0.0    802021
1.0      3700
Name: count, dtype: int64

In [5]:
model.load_model('./pretrain_model/transaction_focus_model_with_public.cbm')
pred_2 = model.predict_proba(new_data[model.feature_names_])[:,1]
new_data['pred_2'] = pred_2
pd.DataFrame(pred_2.round()).value_counts()

0.0    802119
1.0      3602
Name: count, dtype: int64

In [6]:
model.load_model('./pretrain_model/zero_focus_model_with_public.cbm')
pred_3 = model.predict_proba(new_data[model.feature_names_])[:,1]
new_data['pred_3'] = pred_3
pd.DataFrame(pred_3.round()).value_counts()

0.0    802013
1.0      3708
Name: count, dtype: int64

In [11]:
new_data['距離上次刷卡天數'] = new_data['授權日期']-new_data['最後一次交易日期']

In [13]:
new_data['距離上次刷卡天數'].fillna(1,inplace=True)

In [14]:
model.load_model('./pretrain_model/best_model_with_public.cbm')
pred_4 = model.predict_proba(new_data[model.feature_names_])[:,1]
new_data['pred_4'] = pred_4
pd.DataFrame(pred_4.round()).value_counts()

0.0    802149
1.0      3572
Name: count, dtype: int64

In [18]:
new_data['final_pred'] = new_data[['pred_1','pred_2','pred_3','pred_4']].mean(axis=1).round().astype('int8')

In [19]:
new_data['final_pred'].value_counts()

final_pred
0    802107
1      3614
Name: count, dtype: int64

In [21]:
sub.loc[new_data.set_index('交易序號').index,'pred'] = new_data.set_index('交易序號')['final_pred']

In [22]:
sub['pred'].value_counts()

pred
0    802107
1      3614
Name: count, dtype: int64

In [24]:
sub.to_csv('TEAM_4010_submission.csv')

In [8]:
# result_1 = pd.read_csv('./result/01_baseline_with_public_retrain.csv',index_col=0)
# result_2 = pd.read_csv('./result/02_online_pattern_with_public_retrain.csv',index_col=0)
# result_3 = pd.read_csv('./result/03_country_pattern_with_public_retrain.csv',index_col=0)
# result_4 = pd.read_csv('./result/04_shopping_freq_with_public_retrain.csv',index_col=0)
new_result = pd.read_csv('./result/new_sub.csv',index_col=0)
new_result_2 = pd.read_csv('./result/new_sub_2.csv',index_col=0)

result = pd.concat([new_result,new_result_2],axis=1)

# result = pd.concat([result_1,result_2,result_3,result_4,new_result],axis=1)
result['pred_mean'] = result.mean(axis=1)

In [10]:
f1_score(gt,new_result.loc[index,'pred'].round()),f1_score(gt,new_result_2.loc[index,'pred'].round())

(0.8549727035366722, 0.8516068052930057)

In [9]:
f1_score(gt,result.loc[index,'pred_mean'].round())

0.8562618595825428